In [226]:
#pip install selenium
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys #gives the common keyboard keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [227]:
import pandas as pd
import numpy as np
from datetime import datetime  
from time import sleep
import urllib 
from PIL import Image
import openpyxl
from io import BytesIO

In [228]:
import sys
import os
username = os.getlogin()

In [229]:
#Libraries for reading wind speed from image
import re    
#pip install opencv-python
import cv2
#https://towardsdatascience.com/read-text-from-image-with-one-line-of-python-code-c22ede074cac
#pip install pytesseract
import pytesseract
#Download .exe file from https://github.com/UB-Mannheim/tesseract/wiki
#On windows we need to tell python where tesseract is installed
pytesseract.pytesseract.tesseract_cmd = r"C:\Users\{}\AppData\Local\Programs\Tesseract-OCR\tesseract.exe".format(username)

In [230]:
from lxml import etree

In [231]:
import time

In [6]:
#pip install pykml
#from pykml import parser

In [6]:
#import matplotlib.pyplot as plt

## Google Earth Insert Rose

In [244]:
start = time.time()

PATH = r"C:\Program Files (x86)\chromedriver.exe"

kml_path = r"C:\Users\oula.aabkari\Desktop\Nedo_Test\test_kml.kml"

#get the dataframe with: pin name, lats, lons, wind speed, image source
df = kml_parser(kml_path)
df['lats'] = df['lats'].map(coords_converter)
df['lons'] = df['lons'].map(coords_converter)

for i in range(len(df)):   #loop through the coordinate rows    
    
    lat, lon = df['lats'][i], df['lons'][i]
    
    #get the image url of the wind rose and add it to the dataframe
    rose_source = get_rose_from_coords(lat,lon)  
    df.loc[i,'img_src'] = rose_source
    
    #extract the average speed from the image and add it to the dataframe
    avg_speed = get_avg_speed(rose_source)      
    df.loc[i,'avg_speed'] = avg_speed

#write the wind rose images to the kml file
kml_writer(kml_path)
    
print('\nSuccess!')

end = time.time()
seconds = end - start
m, s = divmod(seconds, 60)
print('{} min {} s'.format(round(m),round(s)))

<ipython-input-233-0b4731fe71f7>:11: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(options=options, executable_path=PATH)  #set up the driver



Success!
3 min 22 s


In [200]:
seconds = end - start
m, s = divmod(seconds, 60)
print('{} min {} s'.format(round(m),round(s)))

2 min 43 s


In [245]:
df

,pin_name,lons,lats,avg_speed,img_src
0,5,"(139, 52, 9)","(40, 26, 27)",7.8,https://appraw1.infoc.nedo.go.jp/nedo/winddraw...
1,4,"(139, 52, 2)","(40, 29, 29)",19.0,https://appraw1.infoc.nedo.go.jp/nedo/winddraw...
2,3,"(139, 52, 32)","(40, 32, 47)",7.8,https://appraw1.infoc.nedo.go.jp/nedo/winddraw...
3,2,"(139, 50, 3)","(40, 35, 49)",8.5,https://appraw1.infoc.nedo.go.jp/nedo/winddraw...
4,1,"(139, 52, 25)","(40, 38, 56)",8.5,https://appraw1.infoc.nedo.go.jp/nedo/winddraw...
5,6,"(139, 56, 26)","(40, 34, 48)",1.6,https://appraw1.infoc.nedo.go.jp/nedo/winddraw...
6,7,"(139, 57, 48)","(40, 34, 48)",6.9,https://appraw1.infoc.nedo.go.jp/nedo/winddraw...
7,8,"(139, 58, 52)","(40, 34, 51)",6.9,https://appraw1.infoc.nedo.go.jp/nedo/winddraw...
8,10,"(139, 59, 59)","(40, 34, 21)",15.0,https://appraw1.infoc.nedo.go.jp/nedo/winddraw...
9,9,"(140, 0, 8)","(40, 34, 55)",6.9,https://appraw1.infoc.nedo.go.jp/nedo/winddraw...


## NEDO SCRAPRING

1. Downlaod the chrome web driver from:
https://sites.google.com/chromium.org/driver/

2. Copy the .exe file to C:\Program Files (x86)

In [207]:
#==========View the Progress of the Running Process==============
# for i in range(21):
#     sys.stdout.write('\r')
#     # the exact output you're looking for:
#     sys.stdout.write("Web Scraping Process Is Running: [%-20s] %d%%" % ('='*i, 5*i))
#     sys.stdout.flush()
#     sleep(0.40)
    

#==========================Main===================================

PATH = r"C:\Program Files (x86)\chromedriver.exe"

file_path = "スクリーニングまとめ.xlsx"

coords = get_coords(file_path)  #get the coordinates from xlsx file

wb = openpyxl.load_workbook(file_path)

for i in range(coords.shape[0]):   #loop through the coordinate rows    
    
    #get the image url of the wind rose
    rose_source = get_rose_from_coords(i)  
    
    #extract the average speed from the image
    avg_speed = get_avg_speed(rose_source)         

    #Create a new sheet 'i+1'
    new_sheet = wb.copy_worksheet(wb['【個別】風況＆EADAS'])
    new_sheet.title = str(i+1)

    #Add the image on sheet 'i' Cells A4-A22
    #1/ By saving it
    #image_name = datetime.now().strftime('%d-%m-%Y-%H%M%S.png')
    #urllib.request.urlretrieve(rose_source,image_name)
    #img = openpyxl.drawing.image.Image(image_name)

    #2/ By fetching it from url
    file = BytesIO(urllib.request.urlopen(rose_source).read())
    pil_img = Image.open(file)
    img = openpyxl.drawing.image.Image(pil_img)

    img.width = 430
    img.height = 360
    new_sheet.add_image(img, 'A4')

    wb.move_sheet(new_sheet,offset=-4)

    #Insert wind speed on sheet '一覧' K column
    cell_num = 'K'+str(i+5)
    wb['一覧'][cell_num] = avg_speed

    
#save it as a new file
new_name = datetime.now().strftime('%d-%m-%Y-%H%M%S.xlsx')  
wb.save(new_name)
wb.close()

print('\nSuccess!')

TypeError: get_rose_from_coords() missing 1 required positional argument: 'lon'

In [232]:
#========================Functions=================================

def get_coords(file_path):
    """
    returns a dataframe with latitudes and longitudes 
    """
    coords = pd.read_excel(file_path, sheet_name="一覧",usecols=[2,5],skiprows=[0,1,2,3],header=None,names=['lats','lons'])
    coords.dropna(inplace=True)
    coords = coords.astype('str')
    coords = coords.applymap(lambda s: re.split('[°\'"]',s))
    
    return coords

In [233]:
def get_rose_from_coords(lat,lon):
    """
    Takes the row number from which to read coordinates.
    Returns the Wind Rose URL of the coordinates.
    """
    
    #Set the driver to run headless
    options = Options()
    options.headless = True
    
    driver = webdriver.Chrome(options=options, executable_path=PATH)  #set up the driver
    #driver = webdriver.Chrome(executable_path=PATH)  #set up the driver
    
    driver.get("https://appraw1.infoc.nedo.go.jp/nedo/webgis") 
    
    #Click on 70m 地上高
    elevation = driver.find_element(by='xpath',value="/html/body/div/div[2]/div[2]/div/div/div[1]/map/area[1]")
    elevation.click()
    
    #Click twice on the map
    map1 = driver.find_element(by='xpath', value='//*[@id="contents"]/table/tbody/tr[2]/td[2]/map/area[17]')
    map1.click()

    map2 = driver.find_element(by='xpath', value='//*[@id="contents"]/table/tbody/tr[2]/td[2]/map/area[2]')
    map2.click()

    #Click on 緯度経度 (coords search button)
    search_coords = driver.find_element(by='name', value="navi_btn_kensakull")
    search_coords.click()

    #A new window will open, so switch the driver to that window:
    driver.switch_to.window(driver.window_handles[1])

    #Input coordinates
    x_dgr = driver.find_element(by='name', value="x_dgr")
    x_dgr.send_keys(lon[0])

    x_min = driver.find_element(by='name', value="x_min")
    x_min.send_keys(lon[1])

    x_sec = driver.find_element(by='name', value="x_sec")
    x_sec.send_keys(lon[2])

    y_dgr = driver.find_element(by='name', value="y_dgr")
    y_dgr.send_keys(lat[0])

    y_min = driver.find_element(by='name', value="y_min")
    y_min.send_keys(lat[1])

    y_sec = driver.find_element(by='name', value="y_sec")
    y_sec.send_keys(lat[2])

    #Search for coordinates
    search = driver.find_element(by='xpath', value='/html/body/div/form/table/tbody/tr[3]/td/input[1]')
    search.click()
    
    #Coords window closes, so switch the driver back to the first window
    driver.switch_to.window(driver.window_handles[0])

    #Click on 風配図
    get_rose = driver.find_element(by='name', value='navi_btn_fuhaizu')
    get_rose.click()
    
    #Click on the center of the resulting map
    result_map = driver.find_element(by='xpath', value='//*[@id="jsmap_mapimg2"]')
    w, h = result_map.size['width'], result_map.size['height']
    act = ActionChains(driver)
    #Offsets are relative to the top-left corner of the element.
    #Will set the offset to w/2,h/2 so as to move to the exact center of the elemnt
    #When you call perform(), the events are fired in the order they are queued up
    act.move_to_element_with_offset(result_map,w/2,h/2)
    sleep(5)
    act.click().perform()
    
    #Wait til the action is performed
    try:
        element = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH,'//*[@id="contents"]/table/tbody/tr[2]/td[2]/img')))
    
    finally:
        rose = driver.find_element(by='xpath',value='//*[@id="contents"]/table/tbody/tr[2]/td[2]/img')
        rose_source = rose.get_attribute('src')
        driver.quit()

    return rose_source  

In [234]:
def get_avg_speed(url):
    """
    Converts the Widn Rose url into a cv2 image (numpy array)
    Extracts the average wind speed from the image
    Retunrs it in float.
    """
    req = urllib.request.urlopen(url)
    arr = np.asarray(bytearray(req.read()), dtype=np.uint8)
    img = cv2.imdecode(arr, -1) # 'Load it as it is'
    
    cropped_img = img[:50,400:426]
    text_img = pytesseract.image_to_string(cropped_img,config='--psm 6 --oem 3 -c tessedit_char_whitelist=0123456789.')
    avg_speed = float(re.match(r'(.*)\n',text_img).group(0))
    
    return avg_speed

In [235]:
kml_path = r"C:\Users\oula.aabkari\Desktop\Nedo_Test\My Places.kml"

In [236]:
#Using pyxml
def kml_parser2(kml_path):
    """
    Opens .kml file and gets the coordinates
    outputs a dataframe with pin names and their coordinates
    """
    df = pd.DataFrame(columns=['pin_name','lons','lats','avg_speed','img_src'])

    xmlns = '{http://www.opengis.net/kml/2.2}'

    with open(kml_path,'r') as p:
        root = parser.parse(p).getroot()
        pms = root.findall('.//{}Placemark'.format(xmlns))
        for p in pms:
            if len(p.attrib) == 0:
                df = df.append({'pin_name':p.name.text,
                                'lons':float(p.Point.coordinates.text.split(',')[0]),
                                'lats':float(p.Point.coordinates.text.split(',')[1])},
                               ignore_index=True)
    #df[['lons','lats']] = df['coordinates'].str.split(',',expand=True)[[0,1]]
    df.drop_duplicates(inplace=True)
    
    return df

In [237]:
#Using lxml
def kml_parser(kml_path):
    """
    Opens .kml file and gets the coordinates
    outputs a dataframe with pin names and their coordinates
    """
    df = pd.DataFrame(columns=['pin_name','lons','lats','avg_speed','img_src'])

    xmlns = '{http://www.opengis.net/kml/2.2}'

    with open(kml_path,'r') as myxml:
        root = etree.parse(myxml)
        placemarks = root.findall('.//{}Placemark'.format(xmlns))
        for pm in placemarks:
            if len(pm.attrib) == 0:
                name = pm.findtext('.//{}name'.format(xmlns))
                #name = pm.findtext('{*}name')
                point = pm.find('.//{}Point'.format(xmlns))
                coordinates = point.findtext('.//{}coordinates'.format(xmlns))
                df = df.append({'pin_name':name,
                                'lons':float(coordinates.split(',')[0]),
                                'lats':float(coordinates.split(',')[1])},
                               ignore_index=True)
    #df[['lons','lats']] = df['coordinates'].str.split(',',expand=True)[[0,1]]
    df.drop_duplicates(inplace=True)
    
    return df

In [238]:
def coords_converter(coord):
    """
    Converts the coordinates 
    from float to degs, mins, secs
    """    
    mnt,sec = divmod(coord*3600,60)
    deg,mnt = divmod(mnt,60)
    return(round(deg),round(mnt),round(sec))

In [239]:
#USING PYXML + LXML
def kml_writer2(kml_path):
    """
    Writes the wind rose source image to the kml file
    """
    xmlns = '{http://www.opengis.net/kml/2.2}'

    with open(kml_path,'r') as myxml:
        root = parser.parse(myxml)
        placemarks = root.findall('.//{}Placemark'.format(xmlns))

        for pm in placemarks:
            if len(pm.attrib) == 0:
                name = pm.name
                img_src = df['img_src'][df['pin_name']==name].iloc[0]

                if pm.countchildren() == 5:
                    pm.description._setText(etree.CDATA(pm.description.text +' <img src="{}">'.format(img_src)))
                else:
                    description = etree.SubElement(pm,'description')  
                    description._setText(etree.CDATA('<img src="{}">'.format(img_src)))
        root.write(r"C:\Users\oula.aabkari\Desktop\Nedo_Test\test3.kml",encoding='utf-8')

In [243]:
#Using LXML
def kml_writer(kml_path):
    """
    Writes the wind rose source image to the kml file
    """
    xmlns = '{http://www.opengis.net/kml/2.2}'

    with open(kml_path,'r') as myxml:
        root = etree.parse(myxml)
        placemarks = root.findall('.//{}Placemark'.format(xmlns))

        for pm in placemarks:
            if len(pm.attrib) == 0:
                name = pm.findtext('.//{}name'.format(xmlns))
                
                #Add image source to the description
                img_src = df['img_src'][df['pin_name']==name].iloc[0]
                
                if len(pm.getchildren()) == 5: 
                    description = pm.find('.//{}description'.format(xmlns))
                    description.text = etree.CDATA(description.text +' <img src="{}">'.format(img_src))
                else:
                    description = etree.SubElement(pm,'description')  
                    description.text = etree.CDATA('<img src="{}">'.format(img_src))

                #Change the name to the average speed:
                avg_speed = df['avg_speed'][df['pin_name']==name].iloc[0]
                name = str(avg_speed)

                style = pm.find('.//{}styleUrl'.format(xmlns))

                if avg_speed < 6:
                    style.text = '#m_grn-pushpin'
                elif avg_speed >= 6.5:
                    style.text = '#m_red-pushpin'                 
        
        file_name = datetime.now().strftime('%d-%m-%Y-%H%M%S.kml')
        root.write(file_name,encoding='utf-8')

In [241]:
try:
    if isdir("/mnt/M/"):
        server_base = "/mnt/M/"
    else:
        server_base = "//192.168.2.37/02_ShareFolder_for_Script/"
except:
    server_base = "//192.168.2.37/02_ShareFolder_for_Script/"
    
target_dir = join(server_base, "")

remote_temp_path = "/etc/KML_Map_Processor/Temp_Files"


if len(listdir(target_dir)) > 0:
    for file in listdir(target_dir):
        if isfile(join(target_dir, file)) and file.endswith((".KML", ".kml")):
            print("Processing:", file)
            print()

            get_file = join(target_dir, file)
            put_file = join(remote_temp_path, file)

get_remote_files(get_file,put_file)

NameError: name 'join' is not defined

In [242]:
# GET REMOTE FILES FROM EC2byRHYS TO EC2byME
def get_remote_files(get_file,put_file):
    hostname = '192.168.2.26'
    port = 22
    username = "gpss"
    password = 'gpss2018'
    
    ssh = SSHClient()
    ssh.set_missing_host_key_policy(AutoAddPolicy())

    ssh.connect(hostname=hostname, port=port,username=username, password=password)
    
    sftp = ssh.open_sftp()
    
    sftp.get(get_file,put_file)
    sftp.close()
    ssh.close()

# Useful Links
- URL to PIL Image:
https://stackoverflow.com/questions/7391945/how-do-i-read-image-data-from-a-url-in-python

- np.Array img to PIL Image:
https://stackoverflow.com/questions/10965417/how-to-convert-a-numpy-array-to-pil-image-applying-matplotlib-colormap

# EADAS

# Search On Google

In [8]:
PATH = r"C:\Program Files (x86)\chromedriver.exe"
driver = webdriver.Chrome(PATH)  #set up the driver

<ipython-input-8-ddefbd3e980c>:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)  #set up the driver


In [9]:
driver.get('https://www.google.com/')

#Search for #Types of Chicken on the goofle search bar
search_bar = driver.find_element(by='xpath',value='/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input')
search_bar.clear()  #clear the search bar to make sure that it's empty

search_bar.send_keys("types of chicken")
search_bar.send_keys(Keys.RETURN)  #Keys.RETURN = Enter key

time.sleep(5) #to give us 5sec to see the search results before it closes
#To wait only as long as required check:
#https://selenium-python.readthedocs.io/waits.html

driver.back()    #go back to the previous page <--
driver.forward() # -->

driver.quit()